Sample code for finding ROI (% change of two latest successive funding rounds / time difference).<br>
Put `funding_rounds.csv` here.

In [1]:
# import library
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import *
from collections import Counter
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
# read the CSV
df = pd.read_csv("../bulk_export_processed/funding_rounds.csv")
#df = pd.read_csv("funding_rounds.csv") # place CSV here
# change the cols to keep
keep_col = ['investment_type','raised_amount_usd','org_uuid','announced_on'] # cols for illustration purpose only. choose more for real model
df = df[keep_col]

In [3]:
df.dropna(inplace=True)

no_series_info = (df['investment_type'] == 'undisclosed') | (df['investment_type'] == 'series_unknown')
df.drop(df[no_series_info].index, inplace=True)

df.drop(df[df['raised_amount_usd'].isnull()].index, inplace=True)

In [4]:
df['announced_on'] = pd.to_datetime(df['announced_on'], errors='coerce', format='%d/%m/%Y')

In [5]:
# NEW
# filter by funding round announcement date:  on or after 2010-1-1, by 2020-12-31
# in accordance with company founding date restriction
start_date = dt.datetime(2010, 1, 1)
end_date = dt.datetime(2020, 12, 31)
after_start_date = df["announced_on"] >= start_date
before_end_date = df["announced_on"] <= end_date
df = df.loc[after_start_date & before_end_date]

In [6]:
# df = df[['raised_amount_usd','org_uuid','announced_on']]
# df.head()

In [7]:
df.set_index('announced_on', inplace=True)
df.head()

,investment_type,raised_amount_usd,org_uuid
announced_on,,,
2016-11-06,series_b,15000000.0,8a899f9c-886e-2b9c-378e-49bbeb316cfc
2011-03-31,series_b,1500000.0,31f9d866-3660-5367-8c52-028a7610e441
2010-12-01,series_a,1000000.0,e212a930-25b3-2de3-3c6c-19f0965fcb47
2014-01-09,series_a,1000000.0,e62b140b-2321-d4b5-a74c-19d989be276b
2010-11-16,series_a,4000000.0,20aa45df-3da7-d3e0-719d-1317ddbc0ff3


In [8]:
df['investment_type'].value_counts()

seed                     66726
series_a                 27268
grant                    16567
angel                    13838
series_b                 13200
debt_financing           11570
pre_seed                 11057
post_ipo_equity           6216
series_c                  6017
convertible_note          4864
equity_crowdfunding       3928
private_equity            3119
series_d                  2649
post_ipo_debt             1623
corporate_round           1535
series_e                  1136
product_crowdfunding       969
non_equity_assistance      617
initial_coin_offering      499
series_f                   417
secondary_market           260
series_g                   165
post_ipo_secondary          68
series_h                    66
series_i                    17
series_j                    12
Name: investment_type, dtype: int64

In [9]:
df = df.groupby([pd.Grouper(freq='Q', closed='left'), 'org_uuid']).agg({'investment_type': np.array, 'raised_amount_usd': np.sum})

df
# grp = df.groupby([pd.Grouper(freq = "1H"), "Location"]) \
#       .agg({"Event": np.size, "Cost": np.mean})

investment_type  \
announced_on org_uuid                                                      
2010-03-31   003b7da7-7e0a-1ebe-c32c-19210e9dcbdd         debt_financing   
             003d89d4-47dc-e35a-ddf6-eda9940f3ba7                   seed   
             009e8957-d102-c674-6c9c-98eb11ac8e32               series_a   
             00a4c0e1-c056-4aa7-9262-696dc5192466                  angel   
             00b89e0d-7880-c578-303e-6bd48e371db3               series_a   
...                                                                  ...   
2020-12-31   fff4c94a-af41-4d44-8f42-eb72aa48dac3  initial_coin_offering   
2021-03-31   1a3bd65c-e63e-470c-8ce9-fe435fa2e3d3                   seed   
             23f8d0e6-ee4f-4434-91ac-c8d70ee0fced                   seed   
             6fa4d740-ebc0-e419-f7eb-cc38698b9032                   seed   
             7fe3e86f-6639-4e67-abe3-52e1c5c7e48e      [pre_seed, grant]   

                                                   raised_amount_usd  
announced_on org_uuid                                                 
2010-03-31   003b7da7-7e0a-1ebe-c32c-19210e9dcbdd        260000000.0  
             003d89d4-47dc-e35a-ddf6-eda9940f3ba7           500000.0  
             009e8957-d102-c674-6c9c-98eb11ac8e32          6500000.0  
             00a4c0e1-c056-4aa7-9262-696dc5192466          8000000.0  
             00b89e0d-7880-c578-303e-6bd48e371db3           450000.0  
...                                                              ...  
2020-12-31   fff4c94a-af41-4d44-8f42-eb72aa48dac3          3700000.0  
2021-03-31   1a3bd65c-e63e-470c-8ce9-fe435fa2e3d3           400000.0  
             23f8d0e6-ee4f-4434-91ac-c8d70ee0fced          1091824.0  
             6fa4d740-ebc0-e419-f7eb-cc38698b9032          2000000.0  
             7fe3e86f-6639-4e67-abe3-52e1c5c7e48e           390000.0  

[189126 rows x 2 columns]

In [10]:
df.reset_index(inplace=True)

In [11]:
df

,announced_on,org_uuid,investment_type,raised_amount_usd
0,2010-03-31,003b7da7-7e0a-1ebe-c32c-19210e9dcbdd,debt_financing,260000000.0
1,2010-03-31,003d89d4-47dc-e35a-ddf6-eda9940f3ba7,seed,500000.0
2,2010-03-31,009e8957-d102-c674-6c9c-98eb11ac8e32,series_a,6500000.0
3,2010-03-31,00a4c0e1-c056-4aa7-9262-696dc5192466,angel,8000000.0
4,2010-03-31,00b89e0d-7880-c578-303e-6bd48e371db3,series_a,450000.0
...,...,...,...,...
189121,2020-12-31,fff4c94a-af41-4d44-8f42-eb72aa48dac3,initial_coin_offering,3700000.0
189122,2021-03-31,1a3bd65c-e63e-470c-8ce9-fe435fa2e3d3,seed,400000.0
189123,2021-03-31,23f8d0e6-ee4f-4434-91ac-c8d70ee0fced,seed,1091824.0
189124,2021-03-31,6fa4d740-ebc0-e419-f7eb-cc38698b9032,seed,2000000.0


The code above are new, those below are old

In [12]:
# dt_arr = np.array([(start_date + i * relativedelta(months=+3)).strftime('%Y-%m-%d') for i in range(44)])
# df_test = pd.DataFrame(columns=dt_arr)
# df_test

,2010-01-01,2010-04-01,2010-07-01,2010-10-01,2011-01-01,2011-04-01,2011-07-01,2011-10-01,2012-01-01,2012-04-01,...,2018-07-01,2018-10-01,2019-01-01,2019-04-01,2019-07-01,2019-10-01,2020-01-01,2020-04-01,2020-07-01,2020-10-01


In [13]:
# df.join(df_test).fillna(0, inplace=True)

In [14]:
# NOT USED
"""
# set each time interval as 6 months
# justification: https://about.crunchbase.com/blog/raising-startup-capital/#:~:text=Depending%20on%20the%20type%20of,a%20stake%20in%20the%20startup
# typical funding round duration: 3 months to >1 year
# typical time between rounds: 6 months to 1 year
window_size = 6

In [8]:
# NOT USED
"""
# Helper function 
# Generate windows
num_year = 11 # from 2010 to 2020 (inclusive)
start_year = 2010
def generate_half_year_interval(df):
    all_columns = ['org_uuid']
    for i in range(num_year):
        all_columns += [str(start_year + i) + '_h1']
        all_columns += [str(start_year + i) + '_h2']
        
    df_windows = pd.DataFrame(columns=all_columns)
    df_windows["org_uuid"] = df.drop_duplicates(subset='org_uuid')['org_uuid']
    
    return df_windows
"""

In [4]:
# df.sort_values(by=['org_uuid','raised_amount_usd'])
df.sort_values(by=['raised_amount_usd'])

,investment_type,raised_amount_usd,org_uuid,announced_on
247642,angel,0.000000e+00,3dfb60a7-08bb-63be-8d25-8439c78fc7ff,2018-01-03
303873,series_a,0.000000e+00,3dfb60a7-08bb-63be-8d25-8439c78fc7ff,2020-05-14
342056,pre_seed,0.000000e+00,33962bf0-400d-47f5-b24f-3c6eacbd719a,2020-10-01
269592,pre_seed,0.000000e+00,b6c181b3-299c-6117-1144-9b7689dc5b01,2014-01-16
303251,seed,0.000000e+00,ec3dcecf-e6d0-426c-ba17-d300b685368f,2020-03-09
...,...,...,...,...
210794,series_c,1.400000e+10,74a20af3-f4dd-6188-de60-c4ee6cd0ca4a,2018-06-08
55313,post_ipo_equity,2.127194e+10,e693e2f0-50bb-05ab-8a27-4078f5dacf11,2014-02-28
158549,private_equity,2.180169e+10,44996a96-b3bc-2498-2d15-ea91a4b608e2,2017-03-28
293281,post_ipo_equity,2.400000e+10,7ef1577a-8779-8044-4a42-669b796f8e4f,2020-01-06


In [5]:
# ordinal_investment_type = []

# # perform ordinal encoding: seed/angel = 0, series_a = 1, series_b = 2, series_c = 3
# for investment in df['investment_type']:
#     if investment == 'seed' or investment == 'angel':
#         ordinal_investment_type.append(0)
#     elif investment == 'series_a':
#         ordinal_investment_type.append(1)
#     elif investment == 'series_b':
#         ordinal_investment_type.append(2)
#     elif investment == 'series_c':
#         ordinal_investment_type.append(3)
#     else:
#         ordinal_investment_type.append(99)
        
# df['investment_type'] = ordinal_investment_type

# df.groupby(['investment_type'])

In [6]:
# Here, I convert datetime into number of years
df = df.reset_index(drop=True)
num_data = len(df)
today = ["2021-01-28"] * num_data
df['today'] = pd.DataFrame(today)
df['time'] = (pd.to_datetime(df['today'], errors='coerce', format='%Y-%m-%d') - \
              pd.to_datetime(df['announced_on'], errors='coerce', format='%Y-%m-%d'))


df['time'] = pd.to_numeric(df['time'].dt.days, downcast='integer')
df['time_in_year'] = df['time'].apply(lambda x: round(int(x)/365,4))
df.head()

,investment_type,raised_amount_usd,org_uuid,announced_on,today,time,time_in_year
0,angel,500000.0,df662812-7f97-0b43-9d3e-12f64f504fbb,2004-09-01,2021-01-28,5993,16.4192
1,series_a,12700000.0,df662812-7f97-0b43-9d3e-12f64f504fbb,2005-05-01,2021-01-28,5751,15.7562
2,series_b,27500000.0,df662812-7f97-0b43-9d3e-12f64f504fbb,2006-04-01,2021-01-28,5416,14.8384
3,series_b,10500000.0,f53cb4de-236e-0b1b-dee8-7104a8b018f9,2006-05-01,2021-01-28,5386,14.7562
4,series_b,10000000.0,4111dc8b-c0df-2d24-ed33-30cd137b3098,2007-03-01,2021-01-28,5082,13.9233


In [7]:
df = df.drop(columns = ['announced_on', 'today','time'])

In [8]:
#sort by latest to earliest, then group it
df = df.sort_values(by='time_in_year', ascending=False)
df = df.merge(df.groupby('org_uuid').agg(investment_list=('investment_type',list), raised_amount_usd_list=('raised_amount_usd',list), time_in_year_list = ('time_in_year',list)).reset_index())

In [9]:
# convert to include the latest two data only
#df['latest_two_inv_type'] = df['investment_list'].apply(lambda x: x[-2:])
#df['latest_two_funds_amount'] = df['raised_amount_usd_list'].apply(lambda x: x[-2:])
#df['latest_two_time_in_year'] = df['time_in_year_list'].apply(lambda x: x[-2:])

In [10]:
df.drop(columns=['investment_type', 'raised_amount_usd'], inplace=True)
# df.drop(columns=['investment_type', 'raised_amount_usd', 'announced_on'], inplace=True)
df.drop_duplicates(subset='org_uuid', inplace=True)
#df

In [11]:
investment_list = []

# zip() gives successive difference list
for investment in df['investment_list']:
    # indicate the change as "series_xxx to series_yyy"
    process = [(i + " to " + j) for i, j in zip(investment[: -1], investment[1 :])]
    investment_list.append(process if len(process) else None)

df['investment_delta'] = investment_list

In [12]:
amount_list = []

for amount in df['raised_amount_usd_list']:
    # find change in amount normalized to the smaller amount
    norm_amount_delta = [2 * (j - i)/ (i + j) if (i + j) != 0 else 0 for i, j in zip(amount[: -1], amount[1 :])]
    amount_list.append(norm_amount_delta if len(norm_amount_delta) else None)

df['norm_amount_delta'] = amount_list

In [13]:
time_elapsed_successive = []

for time in df['time_in_year_list']:
    time_elapsed = [(i-j) if len(time) != 1 else 0 for i, j in zip(time[: -1], time[1 :])]
    time_elapsed_successive.append(time_elapsed)
    
df['time_elapsed_successive_two_funds'] = time_elapsed_successive
df = df.reset_index(drop=True)

In [14]:
#include calculation of momentum based on latest two funds, and its change

momentum = []
for y in range(len(df['norm_amount_delta'])):
    if df['norm_amount_delta'].iloc[y] != None:
        change = []
        for j, k in zip(df['norm_amount_delta'].iloc[y][:], df['time_elapsed_successive_two_funds'].iloc[y][:]):
            if k != 0:
                change.append((j)/k)
            else:
                change.append("inf")
    else:
        change = None

    momentum.append(change)
df['momentum'] = momentum

In [15]:
df

,org_uuid,time_in_year,investment_list,raised_amount_usd_list,time_in_year_list,investment_delta,norm_amount_delta,time_elapsed_successive_two_funds,momentum
0,cb8014ea-46f9-2740-f108-e0920fdb2c4b,119.8274,"[grant, grant]","[2050345.0, 53846.0]","[119.8274, 3.9918]",[grant to grant],[-1.8976404708507926],[115.8356],[-0.0163821870897271]
1,b3486553-1d5f-4c90-bc5a-f0018c1dfd3d,117.1562,[pre_seed],[4000000.0],[117.1562],None,None,[],None
2,170e8dd7-dcfb-41e9-8383-d14f572e8bfd,107.7589,"[angel, series_a, series_b, series_c, series_d]","[362757.0, 1611605.0, 3128304.0, 4599259.0, 72...","[107.7589, 6.6356, 5.4247, 4.937, 3.726]","[angel to series_a, series_a to series_b, seri...","[1.2650648665239708, 0.6399696703037969, 0.380...","[101.1233, 1.2109000000000005, 0.4876999999999...","[0.012510122459650453, 0.5285074492557574, 0.7..."
3,14c94b74-c10a-45c5-b569-6f08f35574d2,100.3562,"[pre_seed, pre_seed]","[28211.0, 6957.0]","[100.3562, 0.9397]",[pre_seed to pre_seed],[-1.208712465878071],[99.4165],[-0.012158066979606716]
4,65d0628c-d1d8-4699-9e61-6ef3200839ab,70.4055,"[seed, pre_seed, pre_seed]","[59.0, 33.0, 59613.0]","[70.4055, 1.2986, 0.4082]","[seed to pre_seed, pre_seed to pre_seed]","[-0.5652173913043478, 1.9977869429634845]","[69.10690000000001, 0.8904]","[-0.008178885050615028, 2.2436960275870224]"
...,...,...,...,...,...,...,...,...,...
123363,dc5433a7-5837-42e4-990d-c7f240b69390,0.0849,[series_a],[8600000.0],[0.0849],None,None,[],None
123364,72b016e6-446e-4e37-b220-5b932184f8e3,0.0849,[angel],[20000000.0],[0.0849],None,None,[],None
123365,acd73df8-2c53-403d-8cf1-a7e0d54f7677,0.0849,[seed],[488480.0],[0.0849],None,None,[],None
123366,c88592e1-1a02-4ab2-90ae-4938cd839f11,0.0849,[debt_financing],[1627492.0],[0.0849],None,None,[],None


In [16]:
# # for later use I type the date-related code here first
# date_list = []

# for date in df['date_list']: # assume date is joined into df
#     # find time elapsed between rounds
#     time_delta = [(strptime(j, format='%Y-%m-%d') - strptime(i, format='%Y-%m-%d')) for i, j in zip(date[: -1], date[1 :])]
#     date_list.append(time_delta if len(time_delta) else None)

# df['time_elapsed'] = date_list

In [17]:
# find the duration of the job
# take mean for NaN
#mean = df['time_elapsed'].mean()
#df['time_elapsed'].fillna(value=mean, inplace=True)
# convert TimeDelta to int
#df['time_elapsed'] = df['time_elapsed'].apply(lambda x: x.days)

In [18]:
df.drop(columns=['investment_list', 'raised_amount_usd_list','time_in_year_list', 'time_elapsed_successive_two_funds'], inplace=True)
# df.drop(columns=['investment_list', 'raised_amount_usd_list', 'date_list'], inplace=True)
# for easier checking of results, I dropped rows without delta
#df.dropna(inplace=True)   ** Boris: I think no need to dropna here coz fundings even without delta could be useful
df

,org_uuid,time_in_year,investment_delta,norm_amount_delta,momentum
0,cb8014ea-46f9-2740-f108-e0920fdb2c4b,119.8274,[grant to grant],[-1.8976404708507926],[-0.0163821870897271]
1,b3486553-1d5f-4c90-bc5a-f0018c1dfd3d,117.1562,None,None,None
2,170e8dd7-dcfb-41e9-8383-d14f572e8bfd,107.7589,"[angel to series_a, series_a to series_b, seri...","[1.2650648665239708, 0.6399696703037969, 0.380...","[0.012510122459650453, 0.5285074492557574, 0.7..."
3,14c94b74-c10a-45c5-b569-6f08f35574d2,100.3562,[pre_seed to pre_seed],[-1.208712465878071],[-0.012158066979606716]
4,65d0628c-d1d8-4699-9e61-6ef3200839ab,70.4055,"[seed to pre_seed, pre_seed to pre_seed]","[-0.5652173913043478, 1.9977869429634845]","[-0.008178885050615028, 2.2436960275870224]"
...,...,...,...,...,...
123363,dc5433a7-5837-42e4-990d-c7f240b69390,0.0849,None,None,None
123364,72b016e6-446e-4e37-b220-5b932184f8e3,0.0849,None,None,None
123365,acd73df8-2c53-403d-8cf1-a7e0d54f7677,0.0849,None,None,None
123366,c88592e1-1a02-4ab2-90ae-4938cd839f11,0.0849,None,None,None


In [19]:
# if using our old defined way of ROI, i.e. the momentum between the latest two succesive fundings
df['ROI'] = df['momentum'].apply(lambda x: x[0] if x != None else None)

In [20]:
# find max delta
max_amount_delta_list = []

for delta_list in df['norm_amount_delta']:
    max_amount_delta_list.append(np.array(delta_list).max())

df['max_norm_amount_delta'] = max_amount_delta_list

In [21]:
# find mean delta
mean_amount_delta_list = []

for delta_list in df['norm_amount_delta']:
    mean_amount_delta_list.append(np.array(delta_list).mean() if delta_list != None else None)

df['mean_norm_amount_delta'] = mean_amount_delta_list

In [22]:
# # find ROI
# roi_col = []

# for index, row in df.iterrows():
#     roi_list = np.divide(row['norm_amount_delta'], row['time_elapsed'])
#     roi_col.append(roi_list)

# df['roi_list'] = roi_col

In [23]:
# # find max ROI
# max_roi_list = []

# for roi_list in df['roi_list']:
#     max_roi_list.append(np.array(roi_list).max())

# df['max_roi'] = max_roi_list

In [24]:
# # find mean ROI
# mean_roi_list = []

# for roi_list in df['roi_list']:
#     mean_roi_list.append(np.array(roi_list).mean())

# df['mean_roi'] = mean_roi_list

In [25]:
df.reset_index(inplace=True)
df.drop(axis=1, labels='index', inplace=True)
df

,org_uuid,time_in_year,investment_delta,norm_amount_delta,momentum,ROI,max_norm_amount_delta,mean_norm_amount_delta
0,cb8014ea-46f9-2740-f108-e0920fdb2c4b,119.8274,[grant to grant],[-1.8976404708507926],[-0.0163821870897271],-0.0163822,-1.897640,-1.897640
1,b3486553-1d5f-4c90-bc5a-f0018c1dfd3d,117.1562,None,None,None,None,NaN,NaN
2,170e8dd7-dcfb-41e9-8383-d14f572e8bfd,107.7589,"[angel to series_a, series_a to series_b, seri...","[1.2650648665239708, 0.6399696703037969, 0.380...","[0.012510122459650453, 0.5285074492557574, 0.7...",0.0125101,1.265065,0.682891
3,14c94b74-c10a-45c5-b569-6f08f35574d2,100.3562,[pre_seed to pre_seed],[-1.208712465878071],[-0.012158066979606716],-0.0121581,-1.208712,-1.208712
4,65d0628c-d1d8-4699-9e61-6ef3200839ab,70.4055,"[seed to pre_seed, pre_seed to pre_seed]","[-0.5652173913043478, 1.9977869429634845]","[-0.008178885050615028, 2.2436960275870224]",-0.00817889,1.997787,0.716285
...,...,...,...,...,...,...,...,...
123363,dc5433a7-5837-42e4-990d-c7f240b69390,0.0849,None,None,None,None,NaN,NaN
123364,72b016e6-446e-4e37-b220-5b932184f8e3,0.0849,None,None,None,None,NaN,NaN
123365,acd73df8-2c53-403d-8cf1-a7e0d54f7677,0.0849,None,None,None,None,NaN,NaN
123366,c88592e1-1a02-4ab2-90ae-4938cd839f11,0.0849,None,None,None,None,NaN,NaN


In [26]:
df.to_csv("ROI.csv", index=False)